In [1]:
from xhtml2pdf import pisa
from xhtml2pdf.config.httpconfig import httpConfig

In [2]:
def _convert_html2pdf(source_html, output_path):
    # TODO: make parent directory
    # TODO: check file extension
    httpConfig.save_keys('nosslcheck', True)
    with open(output_path, "w+b") as f:
        pisa_status = pisa.CreatePDF(source_html, dest=f)
        
    return pisa_status.err

In [3]:
table_template = """<table>
  <thead>
    <tr>
      <th>fugafuga</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>hogehoge</th>
    </tr>
  </tbody>
</table>
"""

In [4]:
table_contents_css = """
pdftoc {
    color: #666;
}
pdftoc.pdftoclevel0 {
    font-weight: bold;
    margin-top: 0.5em;
}
pdftoc.pdftoclevel1 {
    margin-left: 1em;
}
pdftoc.pdftoclevel2 {
    margin-left: 2em;
    font-style: italic;
}"""

In [5]:
float_template = """
.box{
 display: inline-block;
 width: 200px;
}

"""

In [130]:
html = f"""<html>
<head>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    
    <style type="text/css">
        <!--
             body {{color: #666666;}}
             {{table_contents_css}}
             table {{border: solid 1px #666666;}}
             {float_template}
             
         -->
    </style>
<title>タイトル</title>
</head>
<body>
<div id="table_of_contents">
<h1>Table of Contents</h1>
<pdf:toc />
</div>
<h1>Hoge</h1>
<div>
test
</div>
{table_template}
<h1>Fuga</h1>
<h2>fugafuga</h2>
<div>
fuga
</div>
<div>
    <div class='box'><img src=images/case=0_trace_num=100_0.png></div>
    <div class='box'><img src=images/case=0_trace_num=100_0.png></div>
    <div class='box'>box</div>
    <div class='after'>after</div>
</div>
</body>
</html>"""

In [131]:
_convert_html2pdf(html, "test_3.pdf")

0

In [133]:
!open test_3.pdf

In [193]:
html

'<html>\n<head>\n    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>\n    \n    <style type="text/css">\n        <!--\n             body {color: #666666;}\n             {table_contents_css}\n             table {border: solid 1px #666666;}\n             \n.box{\n display: inline-block;\n width: 200px;\n}\n\n\n             \n         -->\n    </style>\n<title>タイトル</title>\n</head>\n<body>\n<div id="table_of_contents">\n<h1>Table of Contents</h1>\n<pdf:toc />\n</div>\n<h1>Hoge</h1>\n<div>\ntest\n</div>\n<table>\n  <thead>\n    <tr>\n      <th>fugafuga</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>hogehoge</th>\n    </tr>\n  </tbody>\n</table>\n\n<h1>Fuga</h1>\n<h2>fugafuga</h2>\n<div>\nfuga\n</div>\n<div>\n    <div class=\'box\'>box</div>\n    <div class=\'box\'>box</div>\n    <div class=\'box\'>box</div>\n    <div class=\'after\'>after</div>\n</div>\n</body>\n</html>'

In [7]:
with open("test.html", "w") as f:
    f.write(html)

In [195]:
!open test.html

In [199]:
# State

In [9]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path

import numpy as np

import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.povm import (
    Povm,
    get_x_measurement,
    get_y_measurement,
    get_z_measurement,
)
from quara.objects.qoperation import QOperation
from quara.objects.state import State, get_z0_1q, get_z1_1q, get_x0_1q
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)

In [10]:
%load_ext autoreload
%autoreload 2

In [11]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

povm_x = get_x_measurement(c_sys)
povm_y = get_y_measurement(c_sys)
povm_z = get_z_measurement(c_sys)
povms = [povm_x, povm_y, povm_z]

In [12]:
true_object = get_z0_1q(c_sys)
num_data = [100, 1000]
iteration = 100

# Linear (on_para_eq_constraint=True)
qst = StandardQst(povms, on_para_eq_constraint=True)
estimation_results_affine = data_analysis.estimate(qtomography=qst,
                                                                         true_object=true_object,
                                                                         num_data=num_data,
                                                                         estimator=LinearEstimator(),
                                                                         iteration=iteration)

# Linear (on_para_eq_constraint=False)
qst = StandardQst(povms, on_para_eq_constraint=False)
estimation_results_linear = data_analysis.estimate(qtomography=qst,
                                                                         true_object=true_object,
                                                                         num_data=num_data,
                                                                         estimator=LinearEstimator(),
                                                                         iteration=iteration)

100%|██████████| 100/100 [00:01<00:00, 91.42it/s]


In [13]:
一時保存時のファイル名規則:
{ケース名}_{グラフの種類}_{numの情報}_{グラフの種類固有の識別子}

例: 
等式制約の場合:    
- case=0_trace_num=100
不等式制約の場合
- case=0_eigenvalues_num=100_i=0
- case=0_sum-unphysical-eigenvalues_num=100_greater-than-one
- case=0_sum-unphysical-eigenvalues_num=100_less-than-zero

SyntaxError: invalid syntax (<ipython-input-13-9e831571feb3>, line 1)

In [ ]:
html = """
<h1>Consistency Test</h1>
<hr>
<h1>Asymptotic Consistency Test & Efficiency Test</h1>
# MSEの図
<hr>
<h1>Physicality Violation Test</h1>
<h2>on_para_eq_constraint = True</h2>
# 列 = i = 0, i = 1
# 行 = num_dataの要素分繰り返す
<h2>on_para_eq_constraint = False</h2>
# 列 = :<0, >1
# 行 = num_dataの要素数分繰り返す
"""

In [73]:
report_html = f"""<html>
<head>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    
    <style type="text/css">
        <!--
             body {{color: #666666;}}
             {{table_contents_css}}
             table {{border: solid 1px #666666;}}
             h1 {{margin-top: 60px; border-top: 2px #dcdcdc solid; padding-top: 10px; font-size: 25px}}
             {float_template}
             
         -->
    </style>
<title>タイトル</title>
</head>
<body>
<div id="table_of_contents">
<h1>Table of Contents</h1>
<pdf:toc />
</div>
<h1>MSE</h1>
<div>
(TODO)
</div>
<h1>Physicality Violation Test</h1>
{physicality_violation_test_div}
</body>
</html>"""

In [ ]:

case_id = 0

In [291]:
def _generate_graph_sum_eigenvalues_seq(
    estimation_results: List["EstimationResult"],
    case_id: int,
    true_object,
    num_data: List[int],
) -> list:

    fig_info_list_list = []
    for num_data_index in range(len(num_data)):
        fig_list = physicality_violation_check.make_graphs_sum_unphysical_eigenvalues(
            estimation_results, num_data=num_data, num_data_index=num_data_index,
        )
        fig_info_list = []

        for i, fig in enumerate(fig_list):
            fig_name = f"case={case_id}_sum-unphysical-eigenvalues_num={num_data_index}_type={i}"

            # output
            # TODO
            dir_path = Path(
                "/Users/tomoko/project/rcast/workspace/quara/tutorials/images"
            )
            path = str(dir_path / f"{fig_name}.png")
            fig.update_layout(width=500, height=400)
            dir_path.mkdir(exist_ok=True)
            fig.write_image(path)

            fig_info_list.append(dict(image_path=path, fig=fig, fig_name=fig_name))

        fig_info_list_list.append(fig_info_list)
    return fig_info_list_list


def _generate_sum_eigenvalues_div(fig_info_list_list) -> str:
    graph_block_html_all = ""
    for fig_info_list in fig_info_list_list:
        graph_block_html = ""
        for fig_info in fig_info_list:
            graph_subblock = (
                f"<div class='box'><img src={fig_info['image_path']}></div>"
            )
            graph_block_html += graph_subblock

        graph_block_html_all += f"<div>{graph_block_html}</div>"
    graph_block_html_all = f"<div>{graph_block_html_all}</div>"

    return graph_block_html_all


def generate_sum_eigenvalues_div(
    estimation_results: List["EstimationResult"],
    case_id: int,
    num_data: List[int],
    true_object,
):
    fig_info_list_list = _generate_graph_sum_eigenvalues_seq(
        estimation_results, case_id=case_id, true_object=true_object, num_data=num_data
    )
    div_html = _generate_eigenvalues_div(fig_info_list_list)

    
    return div_html

In [333]:
target_list = [estimation_results_affine, estimation_results_linear]
case_name_list = [
    "LinearEstimator(True)",
    "LinearEstimator(False)"
]

iteration = 100
title = f"Mean Square Value<br>Nrep={iteration}"
mses_list = []

for result in target_list:
    mses, _, _ = data_analysis.convert_to_series(
    result, true_object
)
    mses_list.append(mses)
name_list = [f"case {i}: {name}" for i, name in enumerate(case_name_list)]

fig = data_analysis.make_mses_graph(num_data=num_data, mses=mses_list, names=name_list, title=title)

fig_name = f"mse"

# output
# TODO
dir_path = Path(
                "/Users/tomoko/project/rcast/workspace/quara/tutorials/images"
            )
path = str(dir_path / f"{fig_name}.png")
dir_path.mkdir(exist_ok=True)
fig.write_image(path)

mse_div = f"""<div><img src="{path}"></div>
"""

In [334]:
# 残すこと
case_id = 0
case_name = "LinearEstimator(True)"
target = estimation_results_affine

physicality_violation_test_div_true = f"""
<h2>on_para_eq_constraint=True</h2>
<h3>case {case_id}: {case_name}<h3>
{report.generate_trace_div(target, case_id=case_id, num_data=num_data)}
"""

# Falseの場合
case_id = 1
case_name = "LinearEstimator(False)"
target = estimation_results_linear

physicality_violation_test_div_false = f"""
<h2>on_para_eq_constraint=False</h2>
<h3>case {case_id}: {case_name}<h3>
<h4>Eigenvalue</h4>
{report.generate_eigenvalues_div(target, case_id=case_id, num_data=num_data, true_object=true_object)}
<h4>Sum of unphysical eigenvalues </h4>
{report.generate_sum_eigenvalues_div(target, case_id=case_id, num_data=num_data, true_object=true_object)}
"""

physicality_violation_test_div = physicality_violation_test_div_true + physicality_violation_test_div_false

100%|██████████| 100/100 [00:00<00:00, 7262.74it/s]


In [345]:
report_html = f"""<html>
<head>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    
    <style type="text/css">
        <!--
             body {{color: #666666;}}
             {{table_contents_css}}
             table {{border: solid 1px #666666;}}
             h1 {{margin-top: 60px; border-top: 2px #dcdcdc solid; padding-top: 10px; font-size: 25px}}
             h2 {{font-size: 20px}}
             h3 {{color:#EB9348; font-size: 15px;}}
             .box{{
                 display: inline-block;
                 width: 400px;
                }}
            h4{{font-size: 15px; color: #618CBC; }}
            pdftoc {{
    color: #666;
}}
pdftoc.pdftoclevel0 {{
    font-weight: bold;
    margin-top: 0.5em;
}}
pdftoc.pdftoclevel1 {{
    margin-left: 1em;
}}
pdftoc.pdftoclevel2 {{
    margin-left: 2em;
    font-style: italic;
}}
         -->
    </style>
<title>タイトル</title>
</head>
<body>
<div id="table_of_contents">
<h1>Table of Contents</h1>
<pdf:toc />
</div>
<h1>MSE</h1>
<div>
{mse_div}
<p>TODO: 解析解を追加する</p>
</div>
<h1>Physicality Violation Test</h1>
{physicality_violation_test_div}

<div>
end
</div>
</body>
</html>"""

In [346]:
with open("test.html", "w") as f:
    f.write(report_html)

In [347]:
!open test.html

In [349]:
_convert_html2pdf(report_html, "sample.pdf")

0

In [350]:
!open "sample.pdf"

In [178]:
<div>
    <div class='box'><img src=images/case=0_trace_num=100_0.png></div>
    <div class='box'><img src=images/case=0_trace_num=100_0.png></div>
    <div class='box'>box</div>
    <div class='after'>after</div>
</div>

SyntaxError: invalid syntax (<ipython-input-178-b9f7052db8f0>, line 1)

In [239]:
!pip install psutil

     |████████████████████████████████| 460 kB 3.5 MB/s eta 0:00:01
Using legacy 'setup.py install' for psutil, since package 'wheel' is not installed.
    Running setup.py install for psutil ... done
You should consider upgrading via the '/Users/tomoko/project/rcast/workspace/quara/venv/bin/python3.8 -m pip install --upgrade pip' command.


In [205]:
state.dim

4

In [213]:
get_x0_1q(CompositeSystem([e_sys0])).dim

2

In [129]:
physicality_violation_test_div

'\n<h2>on_para_eq_constraint=True</h2>\n<h3>case 0: LinearEstimator<h3>\n<div><div><img src=images/case=0_trace_num=100_0.png></div><div><img src=images/case=0_trace_num=1000_0.png></div></div>\n'

In [190]:
!ls /Users/tomoko/project/rcast/workspace/quara/tutorials

data
debug_calc_projection.ipynb
images
sandbox_reporting.ipynb
test.html
test.pdf
test_2.pdf
test_3.pdf
turorial_for_MSE_of_povmt.ipynb
turorial_for_plotting_physicality_violation_of_povmt.ipynb
turorial_for_plotting_physicality_violation_of_qst.ipynb
turorial_for_plotting_physicality_violation_of_qst_LinearEstimator.ipynb
turorial_for_plotting_physicality_violation_of_qst_ProjectedLinearEstimator-fix.ipynb
turorial_for_plotting_physicality_violation_of_qst_ProjectedLinearEstimator.ipynb
turorial_for_povmt.ipynb
turorial_for_qst.ipynb
turorial_for_reporting.ipynb
tutorial_for_calc_proj_physical.ipynb
tutorial_for_linear_estimator-debugging.ipynb
tutorial_for_linear_estimator.ipynb
tutorial_for_povmt_mse.ipynb
tutorial_for_simple_lsqpt_1qubit.ipynb
tutorial_for_simple_lsqpt_2qubit.ipynb
tutorial_for_simple_qpt_1qubit.ipynb
tutorial_for_simple_qpt_2qubit.ipynb
